# 01.03.2025

Задание 1
При старте приложения запускаются три потока.
Первый поток заполняет список случайными числами.
Два других потока ожидают заполнения. Когда список
заполнен оба потока запускаются. Первый поток находит
сумму элементов списка, второй поток среднеарифметическое значение в списке. Полученный список, сумма и
среднеарифметическое выводятся на экран. 

In [3]:
import threading
import random
import time

def fill_list(event, lst):
    """ Функция-поток для заполнения списка случайными числами. После заполнения списка устанавливает событие event. """
    for _ in range(10):  # Задаем размер списка
        lst.append(random.randint(1, 100))  # Добавляем случайное число от 1 до 100
    event.set()  # Устанавливаем событие, сигнализируя о завершении заполнения списка

def calculate_sum(lst, event):
    """ Функция-поток для вычисления суммы элементов списка. Ждет установки события event, затем вычисляет сумму. """
    event.wait()  # Ожидаем завершения заполнения списка
    total_sum = sum(lst)
    print(f"Сумма элементов списка: {total_sum}")

def calculate_average(lst, event):
    """ Функция-поток для вычисления среднеарифметического значения элементов списка. Ждет установки события event, затем вычисляет среднее. """
    event.wait()  # Ожидаем завершения заполнения списка
    average = sum(lst) / len(lst)
    print(f"Среднеарифметическое значение: {average:.2f}")

def main():
    lst = []  # Список для хранения случайных чисел
    event = threading.Event()  # Событие для синхронизации потоков

    # Создание и запуск трех потоков
    thread_fill = threading.Thread(target=fill_list, args=(event, lst))
    thread_sum = threading.Thread(target=calculate_sum, args=(lst, event))
    thread_avg = threading.Thread(target=calculate_average, args=(lst, event))

    # Запуск потоков
    thread_fill.start()
    thread_sum.start()
    thread_avg.start()

    # Основной поток ожидает завершения всех потоков
    thread_fill.join()
    thread_sum.join()
    thread_avg.join()

    # Выводим заполненный список
    print(f"Полученный список: {lst}")

if __name__ == "__main__":
    main()

Сумма элементов списка: 403Среднеарифметическое значение: 40.30

Полученный список: [44, 65, 47, 21, 39, 32, 44, 25, 51, 35]


Задание 2
Пользователь с клавиатуры вводит путь к файлу.
После чего запускаются три потока. Первый поток заполняет файл случайными числами. Два других потока
ожидают заполнения. Когда файл заполнен оба потока
стартуют. Первый поток находит все простые числа, второй поток факториал каждого числа в файле. Результаты
поиска каждый поток должен записать в новый файл. На
экран необходимо отобразить статистику выполненных
операций.

In [1]:
import threading
import random
import time
import math

def is_prime(n):
    """ Проверяет, является ли число простым. """
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def factorial(n):
    """ Вычисляет факториал числа. """
    return math.factorial(n)

def generate_random_numbers(filename, event):
    """ Заполняет файл случайными числами. """
    numbers = [random.randint(1, 100) for _ in range(10)]  # Генерация 10 случайных чисел
    with open(filename, 'w') as f:
        f.write("\n".join(map(str, numbers)))
    event.set()  # Сигнализирует другим потокам о завершении заполнения файла

def find_primes(filename, prime_filename, event):
    """ Находит простые числа в файле и сохраняет их в другой файл. """
    event.wait()  # Ожидание завершения заполнения файла
    primes = []
    with open(filename, 'r') as f:
        numbers = map(int, f.read().splitlines())  # Преобразуем строки в целые числа
        for number in numbers:
            if is_prime(number):
                primes.append(number)
    with open(prime_filename, 'w') as f:
        f.write("\n".join(map(str, primes)))  # Сохраняем простые числа в файл

def compute_factorials(filename, factorial_filename, event):
    """ Вычисляет факториалы чисел в файле и сохраняет их в другой файл. """
    event.wait()  # Ожидание завершения заполнения файла
    factorials = []
    with open(filename, 'r') as f:
        numbers = map(int, f.read().splitlines())  # Преобразуем строки в целые числа
        for number in numbers:
            factorials.append(factorial(number))
    with open(factorial_filename, 'w') as f:
        f.write("\n".join(map(str, factorials)))  # Сохраняем факториалы в файл

def display_statistics(numbers, primes, factorials):
    """ Отображение статистики выполненных операций. """
    print(f"Исходные числа: {numbers}")
    print(f"Простые числа: {primes}")
    print(f"Факториалы: {factorials}")

def main():
    input_filename = input("Введите путь к файлу: ")
    prime_output_filename = "primes.txt"
    factorial_output_filename = "factorials.txt"
    event = threading.Event()

    # Создание и запуск потоков
    thread_generate = threading.Thread(target=generate_random_numbers, args=(input_filename, event))
    thread_find_primes = threading.Thread(target=find_primes, args=(input_filename, prime_output_filename, event))
    thread_compute_factorials = threading.Thread(target=compute_factorials, args=(input_filename, factorial_output_filename, event))

    # Запуск потоков
    thread_generate.start()
    thread_find_primes.start()
    thread_compute_factorials.start()

    # Ожидание завершения работы всех потоков
    thread_generate.join()
    thread_find_primes.join()
    thread_compute_factorials.join()

    # Чтение результатов из файлов
    with open(input_filename, 'r') as f:
        numbers = list(map(int, f.read().splitlines()))
    with open(prime_output_filename, 'r') as f:
        primes = list(map(int, f.read().splitlines()))
    with open(factorial_output_filename, 'r') as f:
        factorials = list(map(int, f.read().splitlines()))

    # Отображение статистики
    display_statistics(numbers, primes, factorials)

if __name__ == "__main__":
    main()

Исходные числа: [6, 50, 66, 6, 99, 53, 33, 78, 67, 19]
Простые числа: [53, 67, 19]
Факториалы: [720, 30414093201713378043612608166064768844377641568960512000000000000, 544344939077443064003729240247842752644293064388798874532860126869671081148416000000000000000, 720, 933262154439441526816992388562667004907159682643816214685929638952175999932299156089414639761565182862536979208272237582511852109168640000000000000000000000, 4274883284060025564298013753389399649690343788366813724672000000000000, 8683317618811886495518194401280000000, 11324281178206297831457521158732046228731749579488251990048962825668835325234200766245086213177344000000000000000000, 36471110918188685288249859096605464427167635314049524593701628500267962436943872000000000000000, 121645100408832000]


Задание 3
Пользователь с клавиатуры вводит путь к существующей директории и к новой директории. После чего
запускается поток, который должен скопировать содержимое директории в новое место. Необходимо сохранить
структуру директории. На экран необходимо отобразить
статистику выполненных операций.

In [3]:
import os
import shutil
import threading
import time

In [4]:
def copy_directory(source_dir, destination_dir, event):
    """ Копирует содержимое source_dir в destination_dir, сохраняя структуру подкаталогов. """
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    for root, dirs, files in os.walk(source_dir):
        relative_path = os.path.relpath(root, source_dir)
        new_root = os.path.join(destination_dir, relative_path)

        if not os.path.exists(new_root):
            os.makedirs(new_root)

        for file in files:
            src_file = os.path.join(root, file)
            dst_file = os.path.join(new_root, file)
            shutil.copy(src_file, dst_file)

    event.set()

In [5]:
def main():
    # Получаем пути от пользователя
    source_dir = input("Введите путь к существующей директории: ").strip()
    destination_dir = input("Введите путь к новой директории: ").strip()

    # Проверяем существование исходной директории
    if not os.path.isdir(source_dir):
        print(f"Дирректория '{source_dir}' не найдена или не является дирректорией.")
        return

    # Синхронизация потоков
    event = threading.Event()

    # Запускаем поток для копирования
    copy_thread = threading.Thread(
        target=copy_directory,
        args=(source_dir, destination_dir, event),
    )
    copy_thread.start()

    # Ожидаем завершения работы потока
    event.wait()

    # Выводим статистику
    print(f"Копирование из '{source_dir}' в '{destination_dir}' выполнено.")
    print(f"Количество файлов: {len(os.listdir(destination_dir))}")
    print(f"Количество поддиректорий: {len([d for d in os.listdir(destination_dir) if os.path.isdir(d)])}")

Задание 4
Пользователь склавиатурывводитпутьксуществующей
директории и слово для поиска. После чего запускаются
два потока. Первый должен найти файлы, содержащие
искомое слово и слить их содержимое в один файл. Второй поток ожидает завершения работы первого потока.
После чего проводит вырезание всех запрещенных слов
(список этих слов нужно считать из файла с запрещенными словами) из полученного файла. На экран необходимо
отобразить статистику выполненных операций.

In [6]:
import os
import threading
import re

In [7]:
def search_files_and_merge(directory, word, output_file, event):
    """ Поиск файлов, содержащих указанное слово, и слияние их содержимого в один файл. """
    found_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            filepath = os.path.join(root, file)
            with open(filepath, 'r', encoding='utf-8') as f:
                content = f.read()
                if word.lower() in content.lower():  # Игнорируем регистр при поиске
                    found_files.append(filepath)
                    with open(output_file, 'a+', encoding='utf-8') as out_f:
                        out_f.write(content + '\n')
    print(f"Найдено {len(found_files)} файлов, содержащих слово '{word}'.")
    event.set()  # Оповещаем второй поток о завершении работы

In [8]:
def remove_banned_words(input_file, banned_words_file, output_file):
    """ Удаление запрещенных слов из файла. """
    with open(banned_words_file, 'r', encoding='utf-8') as bw_f:
        banned_words = bw_f.read().splitlines()

    with open(input_file, 'r', encoding='utf-8') as in_f:
        content = in_f.read()

    # Замена запрещенных слов на пустые строки
    for word in banned_words:
        pattern = r'\b' + re.escape(word) + r'\b'
        content = re.sub(pattern, '', content, flags=re.IGNORECASE)

    with open(output_file, 'w', encoding='utf-8') as out_f:
        out_f.write(content)

    print(f"Было удалено {len(banned_words)} запрещенных слов.")

In [9]:
def main():
    directory = input("Введите путь к существующей директории: ").strip()
    word = input("Введите слово для поиска: ").strip()
    output_file = 'merged_files.txt'
    banned_words_file = 'banned_words.txt'
    final_output_file = 'final_output.txt'

    # Синхронизация потоков
    event = threading.Event()

    # Запуск первого потока для поиска и объединения файлов
    search_thread = threading.Thread(
        target=search_files_and_merge,
        args=(directory, word, output_file, event)
    )
    search_thread.start()

    # Ожидание завершения работы первого потока
    event.wait()

    # Запуск второго потока для удаления запрещенных слов
    remove_thread = threading.Thread(
        target=remove_banned_words,
        args=(output_file, banned_words_file, final_output_file)
    )
    remove_thread.start()

    # Ожидание завершения работы второго потока
    remove_thread.join()

    # Вывод статистики
    print(f"Операция выполнена. Объединённый файл находится здесь: {output_file}.")
    print(f"Окончательный файл после удаления запрещённых слов находится здесь: {final_output_file}.")